In [8]:
#import dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import func, inspect

# from sqlalchemy import create_engine
# from sqlalchemy import inspect

In [2]:
# Step 1: Read CSV Files into DataFrames
industry_groups_df = pd.read_csv('Resources/Industry_Groups.csv')
top_ten_df = pd.read_csv('Resources/top_ten_with_industryID.csv')
top_ten_historic_df = pd.read_csv('Resources/top_ten_historic_5y.csv')
fundamental_df_clean = pd.read_csv('Resources/02_ASX_Fundamental_Final_Clean.csv')
industry_groups_df.head()

,industry_name,industry_id
0,Automobiles & Components,1
1,Banks,2
2,Capital Goods,3
3,Class Pend,4
4,Commercial & Professional Services,5


In [ ]:
# #trying to see if i had any null values in 'Date' column of top_ten_historic_df
print(top_ten_historic_df['Ticker'].isnull().sum())

In [ ]:
industry_groups_df.info()

In [ ]:
fundamental_df_clean.info()

In [3]:
#convert datatype to date
top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date'])


In [ ]:
top_ten_historic_df.info()

In [ ]:
#top_ten_historic_df['Date'] = pd.to_datetime(top_ten_historic_df['Date']).apply(lambda x: x.date())

In [ ]:
#print(top_ten_historic_df['Date'].unique())

In [ ]:
industry_groups_df.info()

In [4]:
# Define Database Schema
Base = declarative_base()

class IndustryGroups(Base):
    __tablename__ = 'industry_groups'
    
    industry_name = Column(String)
    industry_id = Column(Integer, primary_key=True)
    

class TopTen(Base):
    __tablename__ = 'top_ten'
    
    ticker = Column(String, primary_key=True)
    company_name = Column(String)
    market_cap = Column(Float)
    industry_id = Column(Integer, ForeignKey('industry_groups.industry_id'))

class TopTenHistoric(Base):
    __tablename__ = 'top_ten_historic'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String, ForeignKey('top_ten.ticker'))
    Date = Column(Date)
    Open = Column(Float)
    High = Column(Float)
    Low = Column(Float)
    Close = Column(Float)
    Adj_Close = Column('Adj Close', Float)
    Volume = Column(Integer)
    
class Fundamental(Base):
    __tablename__ = 'fundamental'
    
    id = Column (Integer, primary_key=True, autoincrement=True)
    Ticker = Column(String,ForeignKey('top_ten.ticker'))
    lastPrice = Column(Float)         
    Change = Column(String)       
    Bid_Ask = Column(String)           
    volumePerDay = Column(Float)         
    volume4wAvg = Column(Float)          
    Open = Column(Float)              
    dayRange = Column(String)              
    prevClose = Column(String)        
    lastTrade = Column(String)            
    oneWeek = Column(String)             
    oneMonth = Column(String)           
    YTD2023 = Column(String)            
    oneYear = Column(String)            
    vsSectorOneYr = Column(String)         
    vsASX200OneYr = Column(String)         
    marketCap = Column(Float)          
    ASXRank = Column(String)            
    sectorRank = Column(String)         
    sharesIssued = Column(Float)       
    Sector = Column(String)               
    similarCompanies = Column(String)     
    EPS = Column(Float)                 
    DPS = Column(Float)                
    bookValuePerShare = Column(Float)    
    Breakdown = Column(String)          
    Recommendation = Column(String)       
    lastUpdated = Column(String)                     
    PE= Column(Float) 
        
#Connect to the Database and Create Tables
# Replace 'sqlite:///asx.db' with your actual database connection string
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
Base.metadata.create_all(engine)




## 1. Job to inspect the tables in the sdlite database

In [9]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [10]:
# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx.db')

In [11]:
# Create inspector object and use it to reflect the database and get a list of table names
inspector = inspect(engine)

# Get a list of table names in the database
table_names = inspector.get_table_names()

# Print the table names
for table_name in table_names:
    print(table_name)

fundamental
industry_groups
top_ten
top_ten_historic


## 2. Job to print all the columsn of each table in the database

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx.db')

# Create an inspector object
inspector = inspect(engine)

# Get a list of table names in the database
table_names = inspector.get_table_names()

# Iterate through the table names and print the columns for each table
for table_name in table_names:
    print(f"Table: {table_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"Column: {column['name']} - Type: {column['type']}")
    print("\n")

Table: industry_groups
Column: industry_name - Type: VARCHAR
Column: industry_id - Type: INTEGER


Table: top_ten
Column: ticker - Type: VARCHAR
Column: company_name - Type: VARCHAR
Column: market_cap - Type: FLOAT
Column: industry_id - Type: INTEGER


Table: top_ten_historic
Column: id - Type: INTEGER
Column: Ticker - Type: VARCHAR
Column: Date - Type: DATE
Column: Open - Type: FLOAT
Column: High - Type: FLOAT
Column: Low - Type: FLOAT
Column: Close - Type: FLOAT
Column: Adj Close - Type: FLOAT
Column: Volume - Type: INTEGER




## 3. Fetch the data from selected table

In [12]:
# SQLAlchemy
from pathlib import Path
from sqlalchemy import create_engine, text


# Create an sqlalchemy engine to connect to the database
engine = create_engine('sqlite:///Resources/top_ten_asx.db')
connection = engine.connect()

In [28]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM industry_groups"))

In [13]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM fundamental LIMIT 5"))

In [ ]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM top_ten LIMIT 5"))

In [35]:
# Query All Records in the the Database - Need to run this line again...
data = connection.execute(text("SELECT * FROM top_ten_historic LIMIT 5"))

In [14]:
# Once this is executed, we need to run the codes again to populate data
for record in data:
    print(record)

(1, 'ARB', 31.26, '1.02 (3.37%)', '$29.88 - $31.35', 160245.0, 245483.0, 31.07, '$30.56 - $31.45', '$30.24', '6:58pm 03/11 (AEDT)', '+3.30%', '+1.20%', '+21.78%', '+11.96%', '+5.78%', '+10.21%', 2570210986.0, '152 of 2,408', '14 of 150', 82220441.0, 'Consumer Cyclical', 'CTD / WEB / SUL', 1.079, 0.62, 0.6, '6 Buy · 5 Hold · 2 Sell', 'Hold', '01/10/23', 28.97126969416126)
(2, 'GUD', 11.08, '0.17 (1.56%)', '$10.77 - $11.35', 366353.0, 441797.0, 10.95, '$10.725 - $11.10', '$10.91', '6:58pm 03/11 (AEDT)', '+2.59%', '-4.24%', '+46.17%', '+37.98%', '+31.80%', '+36.23%', 1561113232.0, '234 of 2,408', '23 of 150', 140894696.0, 'Consumer Cyclical', 'SGR / LNW / SKC', 0.693, 0.39, 0.526, '11 Buy · 2 Hold · 0 Sell', 'Strong Buy', '01/10/23', 15.98845598845599)
(3, 'PWH', 10.12, '0.07 (0.70%)', '$9.96 - $10.92', 37398.0, 127350.0, 10.16, '$10.06 - $10.22', '$10.05', '6:58pm 03/11 (AEDT)', '-0.88%', '-8.08%', '-6.04%', '-3.98%', '-10.17%', '-5.74%', 1016899406.0, '295 of 2,408', '28 of 150', 100484

In [26]:
data.fetchall()

[('Automobiles & Components', 1),
 ('Banks', 2),
 ('Capital Goods', 3),
 ('Class Pend', 4),
 ('Commercial & Professional Services', 5),
 ('Consumer Discretionary Distribution & Retail', 6),
 ('Consumer Durables & Apparel', 7),
 ('Consumer Services', 8),
 ('Consumer Staples Distribution & Retail', 9),
 ('Energy', 10),
 ('Equity Real Estate Investment Trusts (REITs)', 11),
 ('Financial Services', 12),
 ('Food, Beverage & Tobacco', 13),
 ('Health Care Equipment & Services', 14),
 ('Household & Personal Products', 15),
 ('Insurance', 16),
 ('Materials', 17),
 ('Media & Entertainment', 18),
 ('Not Applic', 19),
 ('Pharmaceuticals, Biotechnology & Life Sciences', 20),
 ('Real Estate Management & Development', 21),
 ('Semiconductors & Semiconductor Equipment', 22),
 ('Software & Services', 23),
 ('Technology Hardware & Equipment', 24),
 ('Telecommunication Services', 25),
 ('Transportation', 26),
 ('Utilities', 27)]

In [29]:
data.fetchone()

('Automobiles & Components', 1)

In [60]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

#list all of the tickers found in the top_ten table 
session.query(top_ten.ticker).all()

AttributeError: type object 'Base' has no attribute 'classes'

In [5]:
from sqlalchemy.orm import sessionmaker

#Insert DataFrames into Database
Session = sessionmaker(bind=engine)


session = Session()

# Insert IndustryGroups
session.bulk_insert_mappings(IndustryGroups, industry_groups_df.to_dict(orient="records"))

# Insert TopTen
session.bulk_insert_mappings(TopTen, top_ten_df.to_dict(orient="records"))

# Insert TopTenHistoric
session.bulk_insert_mappings(TopTenHistoric, top_ten_historic_df.to_dict(orient="records"))

# Insert ASX_Fundamental
session.bulk_insert_mappings(Fundamental, fundamental_df_clean.to_dict(orient="records"))


session.commit()




In [47]:
# #Query the Database and Create DataFrame
# # Example: Querying the TopTen table and creating a DataFrame
# top_ten_query = session.query(TopTen).all()
# asx = pd.DataFrame([{
#     'ticker': record.ticker,
#     'company_name': record.company_name,
#     'market_cap': record.market_cap,
#     'industry_id': record.industry_id
# } for record in top_ten_query])

# print(asx)


In [49]:
#import dependencies for inspector
from sqlalchemy import inspect

In [50]:
# Instantiate an Inspector with the engine
inspector = inspect(engine)

In [8]:
# Get a list of table names
table_names = inspector.get_table_names()
print(table_names)

['fundamental', 'industry_groups', 'top_ten', 'top_ten_historic']


In [9]:
#Get columns of top_ten_historic
columns = inspector.get_columns('top_ten')
for column in columns:
    print(column['name'])

ticker
company_name
market_cap
industry_id


In [10]:
#reflect database into ORM classes 
Base = automap_base()
Base.prepare(autoload_with=engine)
Base.classes.keys()

['fundamental', 'top_ten', 'industry_groups', 'top_ten_historic']

In [51]:
#save a reference to the top_ten table as 'top_ten'
top_ten = Base.classes.top_ten

AttributeError: type object 'Base' has no attribute 'classes'

In [12]:
#create a database session object
session = Session(bind =engine)

In [13]:
#list all of the tickers found in the top_ten table 
session.query(top_ten.ticker).all()

[('360',),
 ('4DS',),
 ('A2M',),
 ('AAC',),
 ('ABB',),
 ('ABV',),
 ('AD8',),
 ('AFI',),
 ('AFP',),
 ('AGL',),
 ('AHL',),
 ('AIA',),
 ('AIZ',),
 ('AKP',),
 ('ALD',),
 ('ALL',),
 ('ALQ',),
 ('ALU',),
 ('ALX',),
 ('AMC',),
 ('AMP',),
 ('ANN',),
 ('ANZ',),
 ('APA',),
 ('APE',),
 ('APM',),
 ('ARB',),
 ('ARG',),
 ('ASX',),
 ('AUB',),
 ('AUI',),
 ('AVH',),
 ('AVJ',),
 ('AX1',),
 ('AXE',),
 ('AZJ',),
 ('BAP',),
 ('BEN',),
 ('BFL',),
 ('BGA',),
 ('BGP',),
 ('BHP',),
 ('BIO',),
 ('BKI',),
 ('BLG',),
 ('BOQ',),
 ('BPT',),
 ('BRG',),
 ('BXB',),
 ('BXN',),
 ('CAR',),
 ('CBA',),
 ('CBO',),
 ('CCO',),
 ('CDA',),
 ('CEN',),
 ('CGC',),
 ('CGF',),
 ('CHC',),
 ('CNU',),
 ('COH',),
 ('COL',),
 ('CPU',),
 ('CSL',),
 ('CTD',),
 ('CUV',),
 ('CVW',),
 ('CWP',),
 ('CWY',),
 ('D2O',),
 ('DBI',),
 ('DDR',),
 ('DDT',),
 ('DHG',),
 ('DMP',),
 ('DOW',),
 ('DTL',),
 ('DTZ',),
 ('DXS',),
 ('DY6',),
 ('EBO',),
 ('EDV',),
 ('ELD',),
 ('ERD',),
 ('EVT',),
 ('EXL',),
 ('FBU',),
 ('FLT',),
 ('FMG',),
 ('FPH',),
 ('FRI',),

In [52]:
from sqlalchemy import text

# Conectarse a la base de datos
engine = create_engine('sqlite:///Resources/top_ten_asx.db')

# Consulta para seleccionar algunos registros de la tabla IndustryGroups
query = text("SELECT * FROM industry_groups LIMIT 5")
results = engine.execute(query)
print("\nRegistros en la tabla IndustryGroups:")
for row in results:
    print(row)

AttributeError: 'Engine' object has no attribute 'execute'